In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import scipy.io.wavfile as wav
import os
import pandas as pd
import matplotlib.pyplot as plt
import torchaudio
import glob
import math
import torch.nn as nn
import numpy as np
from bisect import bisect_left
import scipy as sp
import h5py
from scipy.io import wavfile
from glob import glob

import wandb

data_folder = "./datasets/generated_dataset/"
device = 'cuda:0' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu')


In [2]:
config = {
    "dropout" : 0.1,
    "batch_size" : 2, 
    "max_shift" : 500,
    "guess_grid_size" : 100,
    "dataset" : "new_type_dataset_medium.hdf5",
    "cnn_output_size_at_factor_1" : 576,
    "factor" : 10,
    "loss_fn" : "cross_entropy",
    "epochs" : 200,
    "sample_length" : 10000,
    "max_shift" : 100,
    "lr" : 1e-6,
    "n_batch_before_print" : 1,
    "max_freq" : 2500,
    "rir_len" : 1600,
    "rooms_per_batch" : 2,
    "mics_per_batch" : 17,
}

In [3]:
f = h5py.File("../datasets/generated_dataset/" + config["dataset"],"r")
    #f = h5py.File("./datasets/generated_dataset/generated_dataset_medium.hdf5","r")

X = f['input']
y = f['gt']

# f_test = h5py.File("../datasets/generated_dataset/generated_dataset_hard.hdf5","r")
# X_test = f_test['input']
# y_test = f_test['gt']

#X = torch.load(os.path.join(data_folder,"input2.pt"))
#y = torch.load(os.path.join(data_folder,"gt2.pt"))
#X_test = torch.load(os.path.join(data_folder,"input.pt"))
#y_test = torch.load(os.path.join(data_folder,"gt.pt"))

class custom_dataset(Dataset):

    def __init__(self, X, y, idx_min=0,dataset_len=len(X)):
        self.X = X
        self.y = y
        self.dataset_len = dataset_len
        self.idx_min = idx_min

    def __getitem__(self, idx):
        return self.X[idx + self.idx_min],self.y[idx + self.idx_min]

    def __len__(self):
        return self.dataset_len

split_i = int(X.shape[0]*0.9)
dataset = custom_dataset(X,y, 0, split_i)
#dataset_test = custom_dataset(X_test,y_test, 0 ,X_test.shape[0])
dataset_test = custom_dataset(X,y, split_i, X.shape[0] - split_i)
#dataset, dataset_test = torch.utils.data.random_split(dataset, [0.98,0.02])
train_dl = DataLoader(dataset, batch_size=config["batch_size"], shuffle=True)
test_dl = DataLoader(dataset_test, batch_size=config["batch_size"], shuffle=False)

In [4]:

class Block(nn.Module):
    def __init__(self,size):
        super().__init__()
        self.dropout = nn.Dropout(config["dropout"])
        self.l = nn.Linear(size,size)
        self.l2 = nn.Linear(size,size)
        self.act = nn.GELU()
        #self.ln = nn.LayerNorm(size)
        self.apply(self._init_weights)
        
        
        
        

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.002)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.002)
            
    def forward(self, x):
        x = self.dropout(x)
        #return x + self.act(self.l(self.ln(x)))
        return x + self.l2(self.act(self.l(x)))

class Classifier(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.thinker = nn.Sequential(
            #nn.Dropout(dropout),
            Block(10000),
            nn.Linear(10000,config["guess_grid_size"])
            #nn.Dropout(dropout),
            #Block(cnn_output_size_at_factor_1*factor),
            #nn.Dropout(dropout),
            #Block(cnn_output_size_at_factor_1*factor),
        )
        
        self.apply(self._init_weights)
        
        
        
        

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.002)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.002)
        
                
    def forward(self, x):
        
        
        x = self.thinker(x)
        return x
    
model = Classifier().to(device)

In [5]:
if config["loss_fn"] == "cross_entropy":
    loss_fn = torch.nn.CrossEntropyLoss()


def y_to_class_gt(y, guess_grid_size, max_shift) :
    y[y.abs() > max_shift] = max_shift*y[y.abs() > max_shift].sign()

    bin_width = max_shift*2/guess_grid_size
    y = (y/bin_width).round() + guess_grid_size // 2 
    y[y == guess_grid_size] = guess_grid_size - 1
    return y.long()
    

#loss_fn = torch.nn.HuberLoss(delta=10)
def augment_switch(X,y):
    flips = torch.rand(y.shape[0]) < 0.5
    X[flips] = torch.stack([X[flips,:,:,1], X[flips,:,:,0]], dim=3)
    y[flips] = -y[flips]
    return X,y

def augment_shift(X,y, common_shift=True):
    X = torch.complex(X[:,0],X[:,1])

    #augment 1, multiply each of the vectors with phase

    imag_unit = torch.complex(torch.tensor(0.0),torch.tensor(1.0))
    if common_shift:
        sample_shift = (torch.rand(X.shape[0],1,1)*2 - 1)*config["max_shift"]
        phase_shift = (-sample_shift/config["sample_length"]*2*torch.pi*imag_unit*torch.arange(X.shape[1]).unsqueeze(1)).exp()
        new_abf = X*phase_shift
        X = torch.stack([torch.real(new_abf),torch.imag(new_abf)], dim=1)
        
    else:
        sample_shift = (torch.rand(X.shape[0],1,2)*2 - 1)*config["max_shift"]
        phase_shift = (-sample_shift/config["sample_length"]*2*torch.pi*imag_unit*torch.arange(X.shape[1]).unsqueeze(1)).exp()
        new_abf = X*phase_shift
        X = torch.stack([torch.real(new_abf),torch.imag(new_abf)], dim=1)
        y = y + (sample_shift[:,:,0] - sample_shift[:,:,1]).to(int)
    return (X,y)

def augment_amp(X,y):
    amp_max = 3
    amp_min = 0.2
    amp_change_factor = torch.rand(X.shape)*(amp_max - amp_min) + amp_min
    X = X*amp_change_factor
    return (X,y)

def train(dataloader, model, loss_fn, optimizer):
    save_loss_to_file = []
    size = len(dataloader.dataset)
    model.train()
    
    
    print_loss = 0
    for batch, (X,y) in enumerate(dataloader):
        
        # Compute prediction error
        
        
        X = torch.tensor(X).to(device)

        #pull a random sound
        sound_paths = glob("../datasets/reference_sounds/*.wav")
        sound_path = sound_paths[np.random.randint(len(sound_paths))]
        fs,signal = wavfile.read(sound_path)
        start = np.random.randint(0,len(signal) - config["sample_length"] - config["rir_len"]-1, config["rooms_per_batch"])
        # simulate longer sound and then cut to the relevant piece
        signals = np.zeros((config["rooms_per_batch"], config["sample_length"] + config["rir_len"]-1))
        for i in range(config["rooms_per_batch"]):
            signals[i,:] = signal[start[i]:start[i]  + config["sample_length"] + config["rir_len"]-1]
        signals = torch.tensor(signals).to(torch.float32).to(device).unsqueeze(1)


        #plt.plot(np.convolve(X[0],signal))
        #plt.plot(signal + 2*max(signal))

        #torch.nn.functional.conv1d(signal.unsqueeze(1),X.unsqueeze(1))
        #ex1 = torch.fft.rfft(torch.concatenate([torch.zeros(rir_len),torch.ones(sample_length),torch.zeros(rir_len)]))*torch.fft.rfft(X,signal.shape[1]) * torch.fft.rfft(signal)

        q = torch.fft.rfft(torch.fft.irfft(torch.fft.rfft(signals)*torch.fft.rfft(X,signals.shape[1]))[:,:,:config["sample_length"]])[:,:,:config["max_freq"]]
        q = torch.concatenate([torch.concatenate([q,q.roll(i+1, 1)], dim=2) for i in range(config["mics_per_batch"] // 2)],dim=1)
        q = q.view(config["rooms_per_batch"]*(config["mics_per_batch"]*(config["mics_per_batch"] - 1 ))//2, -1)
        q = torch.concatenate([q.real,q.imag],dim=1)
        y = torch.concatenate([y - y.roll(i+1,1) for i in range(config["mics_per_batch"]//2)],dim=1).view(-1)*fs/343
        y = y_to_class_gt(y, config["guess_grid_size"], config["max_shift"]).to(torch.long)
        y = torch.tensor(y).to(device)
        pred = model(q)
        

        loss = loss_fn(pred, y)
        

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print_loss += loss.detach()/config["n_batch_before_print"]
        if batch % config["n_batch_before_print"] == 0:
            if batch == 0:
                print_loss = 0 
                continue
            #wandb.log({"loss":loss})
            loss, current = print_loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            print_loss = 0
            save_loss_to_file.append((loss, current))
    return save_loss_to_file

def test(dataloader, model, loss_fn):
    model.eval() # regression on dropout is not great
    
    with torch.no_grad():
        #model.train()
        print_loss = 0
        counter = 0
        for batch, (X,y) in enumerate(dataloader):
            
            # Compute prediction error
            X = torch.concatenate([X[:,:,:,0], X[:,:,:,1]], dim=1)
            X /= 1e6
            X = X.to(device)
            y = y_to_class_gt(y.squeeze(1), config["guess_grid_size"], config["max_shift"])
            #y = y.to(int).squeeze(1) + guess_grid_size // 2
            y = y.to(device)
            pred = model(X)
            
            

            loss = loss_fn(pred, y)
                
            print_loss += loss.detach()
            counter += 1


        loss = print_loss.item()/counter
        wandb.log({"Test_loss":loss})
        print(f"Test loss: {loss:>7f}")
        print_loss = 0

In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])

In [7]:
for t in range(config["epochs"]):
    losses = np.zeros((0,2))
    print(f"Epoch {t+1}\n-------------------------------")
    #torch.save(model, "model_musan_epoch_" + str(t)+ ".pth")

    #loss_to_file = test(test_dl, model, loss_fn)
    loss_to_file = train(train_dl, model, loss_fn, optimizer)

    
    #losses = np.concatenate([losses,np.array(loss_to_file
    # )])
    #np.save("losses", losses)

#torch.save(model, "./models/" + config["dataset"].split(".")[0]+ "_" + run.name + ".pth")
print("Done!")

Epoch 1
-------------------------------


/tmp/ipykernel_200480/1732332010.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X).to(device)
/tmp/ipykernel_200480/1732332010.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y).to(device)


loss: 236.392990  [    2/   18]
loss: 40.073994  [    4/   18]
loss: 133.630234  [    6/   18]
loss: 16.791235  [    8/   18]
loss: 123.176109  [   10/   18]
loss: 51.577168  [   12/   18]
loss: 219.341675  [   14/   18]
loss: 232.183411  [   16/   18]
Epoch 2
-------------------------------
loss: 66.078537  [    2/   18]
loss: 190.903259  [    4/   18]
loss: 540.439270  [    6/   18]
loss: 44.570305  [    8/   18]
loss: 92.740784  [   10/   18]
loss: 17.559750  [   12/   18]
loss: 92.443390  [   14/   18]
loss: 31.948336  [   16/   18]
Epoch 3
-------------------------------
loss: 423.491943  [    2/   18]
loss: 57.314255  [    4/   18]
loss: 39.926334  [    6/   18]
loss: 63.145840  [    8/   18]
loss: 419.816559  [   10/   18]
loss: 126.815186  [   12/   18]
loss: 88.057343  [   14/   18]
loss: 59.285797  [   16/   18]
Epoch 4
-------------------------------
loss: 22.005112  [    2/   18]
loss: 337.081360  [    4/   18]
loss: 235.015686  [    6/   18]
loss: 35.253273  [    8/   18]


KeyboardInterrupt: 